In [ ]:
import re
import pandas as pd
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [ ]:
cc = pd.read_csv('data/446/bookmarks/DVT446_clamps.csv', usecols=[0,1,2,4,10])
cc['name'] = [re.sub(r'\s+', '', name) for name in cc['name']] # remove whitespace
cc.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'clampsTOH'}, inplace=True)
cc['type'] = ['CableDetectionClamp' if 'CDC' in name else 'CrossCouplingProtector' if 'CCP' in name else 'Centralizer' if 'CENT' in name else 'Other' for name in cc['name']]
cc['hadware_name'] = [name.split('.')[0] for name in cc['name']]
cc['obs'] = [name.split('-')[1] if '-' in name else pd.NA for name in cc['name']]

#cc = cc.loc[cc['type'] == 'OrientationClamp']

# perfs = pd.read_csv('data\mabee2606_perfs_BIASED.csv', usecols=[0,1,7])
# perfs['type'] = 'PERFS'
# perfs.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)

In [ ]:
cc_k = cc.loc[cc['passName'] == 'mainpass_up2_autogain']
cc = cc.loc[cc['passName'] == 'mainpass_up1_autogain']

In [ ]:
# #deleting nv's rows
# cc = cc.loc[cc['obs'] != 'NV']
# cc_nv = cc.loc[cc['obs'] == 'NV']

#set nv's to nan
cc['obs'] = [obs if pd.isna(obs) else pd.NA if obs == 'NV' else obs for obs in cc['obs']]

In [ ]:
features = cc[['type', 'hadware_name', 'depth', 'clampsTOH', 'passName', 'obs']].groupby(['type', 'hadware_name', 'passName', 'obs'], dropna=False).median(numeric_only=True)
features = features.sort_values(by=['depth'], ascending=False).reset_index()

features['fiberTOH'] = [angle+180 if angle < 180 else angle-180 for angle in features['clampsTOH']] # angle for 180deg

features['fiberTOH'] = [angle-270 if angle > 90 and ty=='Centralizer' else angle+270 if ty=='Centralizer' else angle for angle, ty in features[['fiberTOH', 'type']].values]  # angle for 90deg

features['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features['fiberTOH']] # fiver plot angle


# combined = pd.concat([features, perfs])
combined = features
combined['plot_angle'] = [angle-360 if angle > 180 else angle for angle in combined['clampsTOH']] # plot angle
#combined['angle_rounded'] = combined['AngleFromHighSideClockwiseDegrees'].round(0)
#combined['fiber_angle_rounded'] = combined['fiberAngleFromHighSideClockwiseDegrees'].round(0)
combined = combined.round(3)

In [ ]:
fig = go.Figure()

# # Add windows
# gozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))][::-1])

# nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined.loc[combined['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

# fig.update_layout(shapes=[dict(type="path", path=gozone_svg, fillcolor='rgba(154,205,50,0.2)', line=dict(width=0), layer='below'), dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

# Add traces
fig.add_trace(go.Scatter(x=combined.loc[(combined['type']=='CableDetectionClamp') & (pd.isna(combined['obs'])), 'plot_angle'], y=combined.loc[(combined['type']=='CableDetectionClamp') & (pd.isna(combined['obs'])), 'depth'], mode='markers', name='CableDetectionClamp', marker_color='forestgreen', customdata=combined.loc[(combined['type']=='CableDetectionClamp') & (pd.isna(combined['obs'])), ['hadware_name', 'clampsTOH', 'obs']].round(0)))


fig.add_trace(go.Scatter(x=combined.loc[(combined['type']=='CrossCouplingProtector') & (pd.isna(combined['obs'])), 'plot_angle'], y=combined.loc[(combined['type']=='CrossCouplingProtector') & (pd.isna(combined['obs'])), 'depth'], mode='markers', name='CrossCouplingProtector', marker_color='darkolivegreen', customdata=combined.loc[(combined['type']=='CrossCouplingProtector') & (pd.isna(combined['obs'])), ['hadware_name', 'clampsTOH', 'obs']].round(0)))

fig.add_trace(go.Scatter(x=combined.loc[(combined['type']=='CrossCouplingProtector') & (~pd.isna(combined['obs'])), 'plot_angle'], y=combined.loc[(combined['type']=='CrossCouplingProtector') & (~pd.isna(combined['obs'])), 'depth'], mode='markers', name='CrossCouplingProtector NVNH', marker_color='red', customdata=combined.loc[(combined['type']=='CrossCouplingProtector') & (~pd.isna(combined['obs'])), ['hadware_name', 'clampsTOH', 'obs']].round(0)))

fig.add_trace(go.Scatter(x=combined.loc[(combined['type']=='Centralizer') & (pd.isna(combined['obs'])), 'plot_angle'], y=combined.loc[(combined['type']=='Centralizer') & (pd.isna(combined['obs'])), 'depth'], mode='markers', name='Centralizer', marker_color='cadetblue', customdata=combined.loc[(combined['type']=='Centralizer') & (pd.isna(combined['obs'])), ['hadware_name', 'clampsTOH', 'obs']].round(0)))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CableDetectionClamp', 'plot_angle'], y=combined.loc[combined['type']=='CableDetectionClamp', 'depth'], mode='markers', name='CableDetectionClamp', marker_color='forestgreen', customdata=combined.loc[combined['type']=='CableDetectionClamp', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CrossCouplingProtector', 'plot_angle'], y=combined.loc[combined['type']=='CrossCouplingProtector', 'depth'], mode='markers', name='CrossCouplingProtector', marker_color='darkolivegreen', customdata=combined.loc[combined['type']=='CrossCouplingProtector', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='Centralizer', 'plot_angle'], y=combined.loc[combined['type']=='Centralizer', 'depth'], mode='markers', name='Centralizer', marker_color='cadetblue', customdata=combined.loc[combined['type']=='Centralizer', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='Other', 'plot_angle'], y=combined.loc[combined['type']=='Other', 'depth'], mode='markers', name='Other', marker_color='navy', customdata=combined.loc[combined['type']=='Other', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='PERFS', 'plot_angle'], y=combined.loc[combined['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined.loc[combined['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']!='CCableDetectionClamp', 'fiber_plot_angle'], y=combined.loc[combined['type']!='CCableDetectionClamp', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined.loc[combined['type']!='CCableDetectionClamp', ['hadware_name', 'fiberTOH']].round(0)))

fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-270, 270])
fig.update_yaxes(autorange="reversed")
fig.update_layout(hovermode="y", title="Fiber cable orientation main1", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')
fig.show()

In [ ]:
features['type'].value_counts()

In [ ]:
features['obs'].value_counts()

In [ ]:
# features['type'].value_counts()
# O, CCP, C, CDC= features['type'].value_counts()

# pie = make_subplots(rows=1, cols=1, specs=[[{"type": "domain"}]])

# pie.add_trace(go.Pie(
#      values=[CDC, CCP, C, O],
#      labels=['CableDetectionClamp', 'CrossCouplingProtector', 'Centralizer', 'Other'],
#      domain=dict(x=[0, 0.5]),
#      name="Blast Protector",
#      hoverinfo='label+value',
#      sort=False,
#      marker_colors = ['forestgreen', 'darkolivegreen', 'cadetblue', 'navy']
#      ), 
#      row=1, col=1)

# pie.update_layout(title_text="Jewelry type", template="plotly")
# pie.show()

In [ ]:
pio.write_html(fig, file='plotly/446_plot_main1.html', auto_open=False)
# pio.write_html(pie, file='plotly/4532_pie.html', auto_open=False)

In [ ]:
def clamps_overview(clamps, clamps_types):

    fiber = clamps[['type', 'fiber_plot_angle', 'depth', 'hadware_name', 'fiber_angle_rounded']].loc[clamps['type'].isin(clamps_types)]

    fig = go.Figure()
    # # Add windows
    nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(clamps.loc[clamps['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(clamps.loc[clamps['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(clamps.loc[clamps['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(clamps.loc[clamps['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

    fig.update_layout(shapes=[dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

    # Add traces
    for type in clamps['type'].unique():
        fig.add_trace(go.Scatter(x=clamps.loc[clamps['type']==type, 'plot_angle'], y=clamps.loc[clamps['type']==type, 'depth'], mode='markers', name=type, customdata=clamps.loc[clamps['type']==type, ['hadware_name', 'fiber_angle_rounded']]))

    fig.add_trace(go.Scatter(x=fiber['fiber_plot_angle'], y=fiber['depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=fiber[['hadware_name', 'fiber_angle_rounded']]))

    fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
    fig.update_xaxes(range=[-185, 185])
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(hovermode="y", title="Fiber cable orientation overview", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')

    return fig

clamps_overview(combined, ['CableDetectionClamp', 'CrossCouplingProtector', 'Centralizer', 'Other']).show()

In [ ]:
combined.dtypes

In [ ]:
#combined.to_hdf('data/446/446cd.h5', key='cd446', mode='w')
combined.to_pickle('data/446/446cd.pkl')
combined.to_parquet('data/446/446cd.parquet')

kay plot

In [ ]:
features_k = cc_k[['type', 'hadware_name', 'depth', 'AngleFromHighSideClockwiseDegrees', 'passName']].groupby(['type', 'hadware_name', 'passName']).median(numeric_only=True)
features_k = features_k.sort_values(by=['depth'], ascending=False).reset_index()
features_k['fiberAngleFromHighSideClockwiseDegrees'] = [angle+180 if angle < 180 else angle-180 for angle in features_k['AngleFromHighSideClockwiseDegrees']]
features_k['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features_k['fiberAngleFromHighSideClockwiseDegrees']]


# combined = pd.concat([features_k, perfs])
combined_k = features_k
combined_k['plot_angle'] = [angle-360 if angle > 100 else angle for angle in combined_k['AngleFromHighSideClockwiseDegrees']]
combined_k['angle_rounded'] = combined_k['AngleFromHighSideClockwiseDegrees'].round(0)
combined_k['fiber_angle_rounded'] = combined_k['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [ ]:
fig = go.Figure()

# Add windows
gozone_svg = ''.join([f'M {xy[0][0]+30},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+30},{xy[0][1]} ' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-30},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-30},{xy[0][1]}' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))][::-1])

nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined_k.loc[combined_k['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

fig.update_layout(shapes=[dict(type="path", path=gozone_svg, fillcolor='rgba(154,205,50,0.2)', line=dict(width=0), layer='below'), dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

# Add traces
fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'plot_angle'], y=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'depth'], mode='markers', name='CableDetectionClamp', marker_color='forestgreen', customdata=combined_k.loc[combined_k['type']=='CableDetectionClamp', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='CrossCouplingProtector', 'plot_angle'], y=combined_k.loc[combined_k['type']=='CrossCouplingProtector', 'depth'], mode='markers', name='CrossCouplingProtector', marker_color='darkolivegreen', customdata=combined_k.loc[combined_k['type']=='CrossCouplingProtector', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='Centralizer', 'plot_angle'], y=combined_k.loc[combined_k['type']=='Centralizer', 'depth'], mode='markers', name='Centralizer', marker_color='cadetblue', customdata=combined_k.loc[combined_k['type']=='Centralizer', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='PERFS', 'plot_angle'], y=combined_k.loc[combined_k['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined_k.loc[combined_k['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'fiber_plot_angle'], y=combined_k.loc[combined_k['type']=='CableDetectionClamp', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined_k.loc[combined_k['type']=='CableDetectionClamp', ['hadware_name', 'fiber_angle_rounded']]))

fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-270, 100])
fig.update_yaxes(autorange="reversed")
fig.update_layout(hovermode="y", title="Fiber cable orientation main2", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')
fig.show()

In [ ]:
pio.write_html(fig, file='plotly/446_plot_main2.html', auto_open=False)
# pio.write_html(pie, file='plotly/445_pie.html', auto_open=False)

X CHECK

In [ ]:
combined_x = pd.concat([features, features_k])
combined_x['plot_angle'] = [angle-360 if angle > 100 else angle for angle in combined_x['AngleFromHighSideClockwiseDegrees']] # plot angle for rest
combined_x['angle_rounded'] = combined_x['AngleFromHighSideClockwiseDegrees'].round(0)
combined_x['fiber_angle_rounded'] = combined_x['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [ ]:
fig = go.Figure()

# # Add windows
# gozone_svg = ''.join([f'M {xy[0][0]+30},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+30},{xy[0][1]} ' for xy in zip(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-30},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-30},{xy[0][1]}' for xy in zip(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['plot_angle','depth']].values,range(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['plot_angle','depth']].shape[0]))][::-1])

# nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].values,range(combined_x.loc[combined_x['type']=='CableDetectionClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

# fig.update_layout(shapes=[dict(type="path", path=gozone_svg, fillcolor='rgba(154,205,50,0.2)', line=dict(width=0), layer='below'), dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

# Add traces
fig.add_trace(go.Scatter(x=combined_x.loc[(combined_x['type']=='CableDetectionClamp') & (combined_x['passName']=='mainpass_up1_autogain'), 'plot_angle'], y=combined_x.loc[(combined_x['type']=='CableDetectionClamp') & (combined_x['passName']=='mainpass_up1_autogain'), 'depth'], mode='markers', name='CableDetectionClamp main 1', marker_color='blue', customdata=combined_x.loc[(combined_x['type']=='CableDetectionClamp') & (combined_x['passName']=='mainpass_up1_autogain'), ['hadware_name', 'angle_rounded', 'passName']]))

fig.add_trace(go.Scatter(x=combined_x.loc[(combined_x['type']=='CableDetectionClamp') & (combined_x['passName']=='mainpass_up2_autogain'), 'plot_angle'], y=combined_x.loc[(combined_x['type']=='CableDetectionClamp') & (combined_x['passName']=='mainpass_up2_autogain'), 'depth'], mode='markers', name='CableDetectionClamp main 2', marker_color='red', customdata=combined_x.loc[(combined_x['type']=='CableDetectionClamp') & (combined_x['passName']=='mainpass_up2_autogain'), ['hadware_name', 'angle_rounded', 'passName']]))

# fig.add_trace(go.Scatter(x=combined_x.loc[combined_x['type']=='CrossCouplingProtector', 'plot_angle'], y=combined_x.loc[combined_x['type']=='CrossCouplingProtector', 'depth'], mode='markers', name='CrossCouplingProtector', marker_color='darkolivegreen', customdata=combined_x.loc[combined_x['type']=='CrossCouplingProtector', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined_x.loc[combined_x['type']=='Centralizer', 'plot_angle'], y=combined_x.loc[combined_x['type']=='Centralizer', 'depth'], mode='markers', name='Centralizer', marker_color='cadetblue', customdata=combined_x.loc[combined_x['type']=='Centralizer', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined_x.loc[combined_x['type']=='Other', 'plot_angle'], y=combined_x.loc[combined_x['type']=='Other', 'depth'], mode='markers', name='Other', marker_color='navy', customdata=combined_x.loc[combined_x['type']=='Other', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined_x.loc[combined_x['type']=='PERFS', 'plot_angle'], y=combined_x.loc[combined_x['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined_x.loc[combined_x['type']=='PERFS', ['name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined_x.loc[combined_x['type']=='CableDetectionClamp', 'fiber_plot_angle'], y=combined_x.loc[combined_x['type']=='CableDetectionClamp', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined_x.loc[combined_x['type']=='CableDetectionClamp', ['hadware_name', 'fiber_angle_rounded']]))

fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[2]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-240, -100])
fig.update_yaxes(autorange="reversed")
fig.update_layout(hovermode="y", title="x check", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')
fig.show()

In [ ]:
pio.write_html(fig, file='plotly/446_plot_xcheck.html', auto_open=False)